In [3]:
import matplotlib.pyplot as plt
import os
from project_classes import *
from functions import *
from iceflow_library import *
from scipy.optimize import curve_fit
import scipy.optimize as opt

In [4]:
zoom = True
seg_length = 100
season = "2022_Antarctica_BaslerMKB"
flight = "20230127_01"
file_name = "C:\\Users\\rj\\Documents\\cresis_project\\pickle_jar\\layer_export_" + flight + ".pickle"
testing = False

In [5]:
print(f"Force remaking {file_name}...")
mat_pickler_h5py(season, flight, testing_mode=testing)  # make it
layers = read_layers(file_name)  # read in the layers from the pickle file
print(f"File {file_name} created.")

Force remaking C:\Users\rj\Documents\cresis_project\pickle_jar\layer_export_20230127_01.pickle...
Reading data files...
--------------------
f['param']: <HDF5 group "/param" (6 members)>
f['param'].keys(): ['day_seg', 'radar', 'radar_name', 'records', 'season_name', 'sw_version']


layerize_h5py debug:
f['lat'][:]: [[-88.89395645]
 [-88.89382215]
 [-88.89368786]
 ...
 [-88.44666919]
 [-88.44653489]
 [-88.4464006 ]]
f['lat'][:]: -88.8939564477804



layer1: Surface number of points: 96358
layer2: Bottom number of points: 96358
--------------------
C:\Users\rj\Documents\cresis_project\pickle_jar\layer_export_20230127_01.pickle  saved in local directory of this python file.
--------------------

Reading pickle file...
--------------------
Surface
Bottom
--------------------

File C:\Users\rj\Documents\cresis_project\pickle_jar\layer_export_20230127_01.pickle created.


In [13]:
"""
removing brackets from layerize_h5py() lat and lon arrays
"""
# path_segments.append([(layer.lat[i], layer.lon[i]), (layer.lat[i + 1], layer.lon[i + 1])])
"""
Path segment 29200: [(array([-85.95931277]), array([-104.36936099])), (array([-85.95920033]), array([-104.36831876]))]
Layer lat-lon 29200: ([-85.95931277], [-104.36936099])
"""

print(f"Layer lat[0]: {layers[0].lat[0]}")
print(f"Layer lat[0][0]: {layers[0].lat}")

print(type(layers[0].lat[0]))

Layer lat[0]: -89.78842907714407
Layer lat[0][0]: [-89.78842908 -89.78829493 -89.78816077 ... -89.98572386 -89.98585636
 -89.98598917]
<class 'numpy.float64'>


In [14]:
filename = f"C:\\Users\\rj\\Documents\\cresis_project\\pickle_jar\\{season}_{flight}_crossover_points.pickle"

In [18]:
def cross_point(layer, seg_length, quiet=False):
    """
    :param seg_length:
    :param layer: a Layer object
    :param quiet: a boolean to suppress print statements
    :return: the point where the lat-lon crosses over its own path.
    purpose: layers[0].lat and layers[0].lon are numpy arrays of the latitudes and
    longitudes for a flight path. It does not connect back to the beginning.
    This function finds the point where the path crosses over itself.
    The lat-lon of the crossover point will not be exactly the same as the
    lat-lons in the arrays, but it will be very close.
    """
    verbose = not quiet
    print("Finding crossover point...")
    print("--------------------")
    # create a list of line segments of length seg_length
    path_segments = []
    segment_ends = []
    if verbose:
        print(f"Dividing the path into segments of length {seg_length}...")
    for i in range(0, len(layer.lat), seg_length):
        if i + seg_length < len(layer.lat):
            path_segments.append([(layer.lat[i], layer.lon[i]), (layer.lat[i + seg_length], layer.lon[i + seg_length])])
        else:
            path_segments.append([(layer.lat[i], layer.lon[i]), (layer.lat[-1], layer.lon[-1])])

    if verbose:
        print(f"Number of segments: {len(path_segments)}")
    print("Checking for intersections...")
    # check for intersections between the line segments
    # TODO: implement loading bar
    rough_intersections = []
    intersecting_segments = []
    for i in range(len(path_segments)):
        for j in range(i + 1, len(path_segments)):
            if segments_intersect(path_segments[i], path_segments[j]):
                intersection_points = find_segment_intersection(path_segments[i], path_segments[j])
                if intersection_points:
                    rough_intersections.append([intersection_points[0][0], intersection_points[1][0]])
                    intersecting_segments.append([i, j])
                    if verbose:
                        print(f"Segments {i} and {j} intersect near "
                              f"({intersection_points[0][0]}, {intersection_points[1][0]})")

    if verbose:
        print("\nChecking for a more precise intersection...")
    fine_intersections = []
    intersection_indices = []

    printed = False

    for seg_pair in range(len(intersecting_segments)):
        seg1 = intersecting_segments[seg_pair][0]
        seg2 = intersecting_segments[seg_pair][1]

        # translate segment numbers to indices in the lat-lon arrays
        if seg1 == 0:
            seg1_start = 0
            seg1_end = seg_length
        else:
            seg1_start = seg1 * seg_length
            seg1_end = seg1_start + seg_length
        if seg2 == 0:
            seg2_start = 0
            seg2_end = seg_length
        else:
            seg2_start = seg2 * seg_length
            seg2_end = seg2_start + seg_length
        if verbose:
            print(f"segment {seg1} start: {seg1_start}, segment 1 end: {seg1_end}")
        if verbose:
            print(f"segment {seg2} start: {seg2_start}, segment 2 end: {seg2_end}")

        # create a list of line segments of length 1
        path_segments = []
        for i in range(seg1_start, seg1_end):
            path_segments.append([(layer.lat[i], layer.lon[i]), (layer.lat[i + 1], layer.lon[i + 1])])
            # if not printed:
                # print(f"Path segment {i}: {path_segments[-1]}")
                # print(f"Layer lat-lon {i}: ({layer.lat[i]}, {layer.lon[i]})")
                # printed = True
        for i in range(seg2_start, seg2_end):
            path_segments.append([(layer.lat[i], layer.lon[i]), (layer.lat[i + 1], layer.lon[i + 1])])

        # check for intersections between the line segments
        for first_seg in range(len(path_segments)):  # compare each segment
            for sec_seg in range(first_seg + 1, len(path_segments)):  # to every other segment after it
                if segments_intersect(path_segments[first_seg], path_segments[sec_seg]):  # if they intersect
                    intersection_points = find_segment_intersection(path_segments[first_seg], path_segments[sec_seg])  # find the intersection
                    if intersection_points:
                        index1 = seg1_start + first_seg
                        index2 = seg2_start + sec_seg
                        # intersections.append([intersection_points])
                        segment_ends.append([[path_segments[first_seg][0], path_segments[first_seg][1], index1],
                                             [path_segments[sec_seg][0], path_segments[sec_seg][1], index2]])

                        fine_intersections.append([intersection_points[0][0], intersection_points[1][0]])
                        # fine_intersections are the first two points of the segment_ends list, i.e. two endpoints on
                        # opposing legs of the X. segment_ends are all four points of the X.
                        if verbose:
                            print(f"Segments {seg1} and {seg2} intersect near indices "
                                  f"{index1} and {index2}\nThis corresponds roughly to the "
                                  f"lat-lon: ({fine_intersections[-1][0]}, {fine_intersections[-1][1]})")
                        intersection_indices.append([index1, index2])

                        if not printed:
                            print("--------------------\n\n")
                            print(f"path_segments[first_seg]: {path_segments[first_seg]}")
                            print(f"path_segments[sec_seg]: {path_segments[sec_seg]}\n")
                            print(f"segment_ends: {segment_ends}\n")

                            print("--------------------\n\n")
                            printed = True

    print(f"Number of intersections: {len(fine_intersections)}")
    if verbose:
        print(f"Number of rough intersections: {len(rough_intersections)}")
        print(f"Number of intersection indices: {len(intersection_indices)}")
        # print(f"Indices: {intersect_indices}")
        # for index in intersect_indices:
        #     print(f"Index: {index}: ")
    # for i in range(len(intersection_indices)):
    #     print(f"Index {i}: \n"
    #           f"indices: \t{intersection_indices[i]}\n"
    #           f"lat-lon: \t({fine_intersections[i][0]}, {fine_intersections[i][1]})\n"
    #           f"segment ends: \t{segment_ends[i]}\n"
    #           f"lat-lon by layer: \t({layer.lat[intersection_indices[i][0]]}, {layer.lon[intersection_indices[i][0]]})")

    print(f"Intersection at index {intersection_indices[0][0]} and {intersection_indices[0][1]}")
    for i in range(len(intersection_indices)):
        # TODO Error 1: correct for the fine intersection points being wrong for some reason
            # (lat is close but lon is 6 degrees off in the 20181112_02 flight)
        fine_intersections[i][0] = layer.lat[intersection_indices[i][0]]
        fine_intersections[i][1] = layer.lon[intersection_indices[i][0]]
    print("--------------------\n")

    return fine_intersections, intersection_indices, segment_ends

intersection_points, intersection_indices, segment_ends = cross_point(layers[0], seg_length, quiet=True)

Finding crossover point...
--------------------
Checking for intersections...
--------------------


path_segments[first_seg]: [(-85.95336842784279, -104.31383823598436), (-85.95325594062754, -104.31279852896284)]
path_segments[sec_seg]: [(-85.95332237124342, -104.31346984061088), (-85.95339582068333, -104.31187657863418)]

segment_ends: [[[(-85.95336842784279, -104.31383823598436), (-85.95325594062754, -104.31279852896284), 29253], [(-85.95332237124342, -104.31346984061088), (-85.95339582068333, -104.31187657863418), 63621]]]

--------------------


Number of intersections: 6
Intersection at index 29253 and 63621
--------------------

